In [ ]:
import csv
import random
import math
from csv import reader
import numpy as np

#leer archivo
def leer_csv(archivo):
	dataset = list()
	with open(archivo, 'r') as file:
		for fila in reader(file):
			if not fila:
				continue
			dataset.append(fila)
	return dataset

# convertir datos a flotantes
def dato_a_float(dataset, columna):
	for fila in dataset:
		fila[columna] = float(fila[columna].strip())

# Convertir valores a numericos
def dato_a_num(dataset, columna):
  vals_clase = [fila[columna] for fila in dataset]
  unico = set(vals_clase)
  buscar = {}
  for i, valor in enumerate(unico):
    buscar[valor] = i
  for fila in dataset:
    fila[columna] = buscar[fila[columna]]

#Devuelve la frecuencia de las clases
def frec_train(vals_pred):
  frec_valor = {}
  for valor in vals_pred:
    if valor in frec_valor:
      frec_valor[valor] += 1
    else:
      frec_valor[valor] = 1
  return frec_valor

#Devuelve el dataset dividido (train, test)
def divide_Datos(dataset, factor_split_datos):
	tamanoTrain = int(len(dataset) * factor_split_datos)
	datosTrain = []
	datosTest = list(dataset)
	while len(datosTrain) < tamanoTrain:
		index = random.randrange(len(datosTest))
		datosTrain.append(datosTest.pop(index))
	return [datosTrain, datosTest]

def separar_clase_training(train_data):
  clase_datos = dict()
  for i in range(len(train_data)):
    for k in range(len(train_data[0]) - 1):
      if k not in clase_datos:
        clase_datos[k] = list()
      clase_datos[k].append((train_data[i][k], train_data[i][-1]))
  return clase_datos

def probabilidades_atributos(separate_training_data,frec):
  prob_atributos = dict()
  for k, v in separate_training_data.items():
    for i in v:
      if k not in prob_atributos:
        prob_atributos[k] = list()
      proba = (i, v.count(i) / frec[i[1]])
      if proba not in prob_atributos[k]:
        prob_atributos[k].append(proba)
  return prob_atributos

#Predecir
def predecir(vals_training,valsTest,frecTrain,vals_pred):
  test={}
  predicciones=[]
  for vals in frecTrain:
    predicciones.append([])
  for clase in frecTrain:
    for instancia in valsTest:
      prediccionest_prob=1
      test_instancia = instancia[:-1]
      for k, v in vals_training.items():
        test[k]=(test_instancia[k],clase)
        for i in range(len(v)):
          if v[i][0] == test[k]:
            prediccionest_prob *= v[i][1]
      predicciones[clase].append(prediccionest_prob)
  #Acomodar las prob. prediccionesterioris
  predicciones_np=np.array([np.array(xi) for xi in predicciones])
  #Determinar el mayor y su indice
  max=np.argmax(predicciones_np, axis=0)
  max=max.tolist()
  #mandar el dato predicho con el dato esperado
  return np.column_stack((max, vals_pred))

#Accuracy
def accuracy(predicciones):
  correctos = 0
  #comparar dato predicho con dato esperado
  for resultado in predicciones:
    if resultado[0] == resultado[1]:
      correctos += 1
  return correctos / len(predicciones) * 100

#Comienza la ejecucion
#archivo = 'pima-indians-diabetes_discrete.csv'
#archivo = 'iris(prueba).csv'
archivo = 'iriscsv.csv'
#archivo = 'iris.datadiscrete(2).csv'
dataset = leer_csv(archivo)
dataset2 = leer_csv(archivo)
#convertir datos a numericos
for i in range(len(dataset[0])):
	dato_a_num(dataset, i)

factor_split = 2/3

#dividir los datasets
datosTrain, datosTest = divide_Datos(dataset, factor_split)
#print('Total = {0} filas. Entrenamiento = {1} y evaluación = {2}'\
#      .format(len(dataset), len(datosTrain), len(datosTest)))

# preparar modelo
#Valores objetivo
vals_pred = [fila[len(datosTest[0])-1] for fila in datosTest]
#Frecuencia de las clases de entrenamiento
frecTrain=frec_train(vals_pred)
#Entrenamiento
clases_train=separar_clase_training(datosTrain)
prob_atribs=probabilidades_atributos(clases_train,frecTrain)

#Prediccion
prediccion = predecir(prob_atribs, datosTest,frecTrain,vals_pred)
#Accuracy
print("\nAccuracy: " + str(accuracy(prediccion)) + "%")

Total = 150 filas. Entrenamiento = 100 y evaluación = 50

Accuracy: 0.0%
